<table style="margin: auto; background-color: white;">
    <tr>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
    </tr>
</table>

#TUTORIAL 1

Bem-vindo! Neste tutorial você aprenderá sobre a interface de programação da plataforma **Flautim** e também como montar um experimento simples de classificação usando o dataset [IRIS](https://archive.ics.uci.edu/dataset/53/iris).

O código desse tutorial pode ser acessado em: [clique aqui.](https://github.com/FutureLab-DCC/flautim_tutoriais/tree/main/TUTORIAL_1)


Vamos começar entendendo a interface de programação da **Flautim**, representada na figura abaixo. A **Flautim_api** é uma biblioteca modularizada que facilita a realização de experimentos de aprendizado de máquina, seja convencional/centralizado ou federado.

Todo projeto **Flautim** precisa herdar essa biblioteca, que contém submódulos específicos para diferentes tecnologias (por exemplo, submódulos para PyTorch, TensorFlow, etc). Neste tutorial usaremos o submódulo para PyTorch.

<div style="text-align: center;"> <table style="margin: auto;"> <tr> <td> <img src='https://drive.google.com/uc?export=view&id=1QOI4jWrwS979xhW_wlGzkPa2bMA-giuc' alt="Interface da plataforma Flautim" width="800" /> </td> </tr> </table> </div>


Dentro de cada submódulo existem três componentes principais (classes):

**1. Dataset:** é utilizado para representar os dados do experimento. Esta classe pode ser reutilizada em diversos experimentos e com diferentes modelos, sendo o componente mais versátil e reutilizável. Os usuários podem importar os dados de diversas fontes, como arquivos locais ou bases de dados online, desde que a classe Dataset seja herdada.

**2. Model:** representa qualquer conjunto de parâmetros treináveis dentro do projeto. Ela permite a aplicação de técnicas de aprendizado de máquina por meio de treinamento desses parâmetros. No caso de PyTorch, a classe herda a nn.Module, que define a estrutura e os parâmetros treináveis do modelo.

**3. Experiment:** define o ciclo de treinamento e validação. Existem dois tipos principais de experimentos: o experimento centralizado, que segue o fluxo
convencional de aprendizado de máquina, e o experimento federado, adaptado para
aprendizado federado. Esta classe inclui duas funções principais, um loop de
treinamento e um loop de validação, que realizam a atualização dos parâmetros e
cálculo das métricas de custo, respectivamente.

Além desses três componentes principais, há também um módulo chamado Common. Este módulo fornece acesso a classes essenciais para o gerenciamento de dados e monitoramento do treinamento.


Com essa visão geral, você está pronto para começar montar seus próprios experimentos. Vamos ao passo a passo!

###Passo 1: Criando o dataset que será usado no experimento

Um conjunto de dados no Flautim é acessado por um arquivo .py que deve conter uma classe que herda de Dataset.

**Exemplo: Implementando a Classe IRISDataset**

O código abaixo implementa uma classe IRISDataset utilizando o dataset IRIS para resolver um problema de classificação.

In [ ]:
from flautim2.pytorch.Dataset import Dataset
import torch
import copy

class IRISDataset(Dataset):

    def __init__(self, file, **kwargs):
        super(IRISDataset, self).__init__(name = "IRIS", **kwargs)

        # Defina o que são features e targets
        self.features = file.iloc[:, 0:4].values
        self.target = file.iloc[:, 4].values

        # Número de amostras para teste
        self.test_size = int(0.2 * len(file))

        # Defina o tipo do tensor de entrada e de saída.
        self.xdtype = torch.float32
        self.ydtype = torch.int64

        # batch_size
        self.batch_size = 10

        # shuffle
        self.shuffle = True

        # num_workers
        self.num_workers = 1

    def train(self) -> Dataset:
        # Separação das amostras para treino
        self.features = self.features[:-self.test_size]
        self.target = self.target[:-self.test_size]
        return copy.deepcopy(self)

    def validation(self) -> Dataset:
        # Separação das amostras para validação
        self.features = self.features[-self.test_size:]
        self.target = self.target[-self.test_size:]
        return copy.deepcopy(self)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.LongTensor([self.target[idx]])

###Passo 2: Criando o modelo que será usado no experimento

Agora, vamos criar a classe que implementa o modelo. Essa classe deve herdar da classe Model.


**Exemplo: Implementando a Classe IRISModel**

A classe IRISModel implementa uma rede neural com 4 entradas e 3 saídas.

In [ ]:
from flautim2.pytorch.Model import Model
import torch

class IRISModel(Model):
    def __init__(self, context, num_classes = 3, **kwargs):
        super(IRISModel, self).__init__(context, name = "IRIS-NN", **kwargs)

        # Rede neural com 4 entradas e 3 saídas
        self.c1 = torch.nn.Linear(4, 10)
        self.c2 = torch.nn.dropout(0.1)
        self.c2 = torch.nn.Linear(10, num_classes)


    def forward(self, x):
        x = torch.relu(self.c1(x))
        x = torch.relu(self.c2(x))
        return x

###Passo 3: Criando o experimento

Por fim, será criado o experimento, isto é, uma classe que implementa os loops de treinamento e validação do modelo IRISModel no dataset IRISDataset. Para isso, precisamos criar dois arquivos .py, o run.py (que deve ter obrigatoriamente esse nome) e o .py responsável por implementar o experimento, descritos a seguir:

**1. Arquivo run.py:**

* Esse arquivo é o ponto de entrada de todo experimento Flautim, pois é ele
que deve iniciar a classe do experimento e também um modelo e um Dataset.

**2. Arquivo .py do experimento:**

* Esse arquivo deve conter uma classe que implemente os métodos de treinamento (training_loop) e validação (evaluation_loop) do modelo. Essa classe deve herdar da classe Experiment.

Esse tutorial cobrirá dois tipos de experimentos, um experimento centralizado e outro descentralizado. Portanto, o passo 3 será dividido entre esses dois cenários.

####Passo 3.1: Experimento centralizado

**Implementando a Classe IRISExperiment**

No código abaixo, a classe IRISExperiment foi criada no modo centralizado com seus métodos training_loop e evaluation_loop para treinar e testar a rede neural. Esses métodos retornam o valor da função de perda e a acurácia de treinamento e de validação.

In [ ]:
#Chamar flautim.pytorch.centralized.Experiment para experimento centralizado
from flautim.pytorch.centralized.Experiment import Experiment

import flautim2 as fl
import numpy as np
import torch
import time

class IRISExperiment(Experiment):
    def __init__(self, model, dataset, context, **kwargs):
        super(IRISExperiment, self).__init__(model, dataset, context, **kwargs)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)
        self.epochs = kwargs.get('epochs', 30)


    def training_loop(self, data_loader):
        self.model.train()
        error_loss = 0.0
        yhat, y_real = [], []

        for X, y in data_loader:
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()

            error_loss += loss.cpu().item()
            _, predicted = torch.max(outputs.data, 1)
            yhat.append(predicted.detach().cpu())
            y_real.append(y.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)

        return float(error_loss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

    def validation_loop(self, data_loader):
        error_loss = 0.0
        self.model.eval()

        with torch.no_grad():
            for X, y in data_loader:
                outputs = self.model(X)
                error_loss += self.criterion(outputs, y).item()
                _, predicted = torch.max(outputs.data, 1)
                yhat.append(predicted.detach().cpu())
            y_real.append(y.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)

        return float(error_loss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

**Implementando o run.py para realização de um experimento centralizado**

**1. Upload do Conjunto de Dados:**

* *Arquivo Local:* Se o seu conjunto de dados for um arquivo (por exemplo, CSV, NPZ, etc.), faça o upload para a plataforma e carregue-o usando o caminho ./data/nomedoarquivo.

* *URL:* Se o conjunto de dados estiver disponível em uma URL, inclua a URL no seu código e carregue-o diretamente.

**2. Crie uma instância para IRISDataset, IRISModel, IRISExperiment.**

**3. Execute as funções:**

* ***experiment.run:*** Executa o experimento centralizado.

In [ ]:
import flautim as fl
import IRISDataset, IRISModel, IRISExperiment
import numpy as np
import pandas as pd
import flautim2.metrics as flm


if __name__ == '__main__':

    context = fl.init()

    fl.log(f"Flautim2 inicializado!!!")

    # Carregue os dados usando dataset próprio
    iris = pd.read_csv("./data/iris.csv", header=None)

    # Carregue os dados usando uma URL
    iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
    iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    iris['class'] = pd.factorize(iris['class'])[0]

    # Embaralhe os dados
    file = iris.sample(frac=1, random_state=42).reset_index(drop=True)

    dataset = IRISDataset.IRISDataset(file, batch_size = 10, shuffle = False, num_workers = 0)

    model = IRISModel.IRISModel(context)

    experiment = IRISExperiment.IRISExperiment(model, dataset, context)

    # Exemplo de métrica implementada pelo usuário
    def accuracy_2(y, y_hat):
        y = np.asarray(y)
        y_hat = np.asarray(y_hat)
        return np.mean(y == y_hat)

    # Adiciona a métrica ao módulo de métricas
    flm.Metrics.accuracy_2 = accuracy_2

    experiment.run(metrics = {'ACCURACY': flm.Metrics.accuracy, 'ACCURACY_2': flm.Metrics.accuracy_2})

####Passo 3.2: Experimento federado
**Implementando a Classe IRISExperiment**

No código abaixo, criamos a classe IRISExperiment no modo federado com seus métodos training_loop e evaluation_loop para treinar e testar a rede neural. Esses métodos retornam o valor da função de perda e a acurácia de treinamento e de validação.

In [ ]:
#Chamar flautim.pytorch.federated.Experiment para experimento federado
from flautim.pytorch.federated.Experiment import Experiment

import flautim.metrics as flm
import numpy as np
import time
import torch

class IRISExperiment(Experiment):
    def __init__(self, model, dataset, context, **kwargs):
        super(IRISExperiment, self).__init__(model, dataset, context, **kwargs)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.01)
        self.epochs = kwargs.get('epochs', 20)

    # Exemplo de métrica implementada pelo usuário
    def accuracy_2(y, y_hat):
        y = np.asarray(y)
        y_hat = np.asarray(y_hat)
        return np.mean(y == y_hat)

    # Adiciona a métrica ao módulo de métricas
    flm.Metrics.accuracy_2 = accuracy_2


    def training_loop(self, data_loader):

        self.model.train()
        error_loss = 0.0
        yhat, y_real = [], []

        for X, y in data_loader:

            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()

            error_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            yhat.append(predicted.detach().cpu())
            y_real.append(y.detach().cpu())

        accuracy = flm.Metrics.accuracy(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = flm.Metrics.accuracy_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)
        return error_loss, {"ACCURACY": accuracy, "ACCURACY_2": accuracy_2}

    def validation_loop(self, data_loader):
        error_loss = 0.0
        yhat, y_real = [], []
        self.model.eval()

        with torch.no_grad():
            for X, y in data_loader:
                outputs = self.model(X)
                error_loss += self.criterion(outputs, y).item()
                _, predicted = torch.max(outputs.data, 1)
                yhat.append(predicted.detach().cpu())
                y_real.append(y.detach().cpu())

        accuracy = flm.Metrics.accuracy(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = flm.Metrics.accuracy_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        error_loss = error_loss / len(data_loader)
        return error_loss, {"ACCURACY": accuracy, "ACCURACY_2": accuracy_2}

**Implementando o run.py para realização de um experimento federado**

**1. Upload do Conjunto de Dados:**

* *Arquivo Local:* Se o seu conjunto de dados for um arquivo (por exemplo, CSV, NPZ, etc.), faça o upload para a plataforma e carregue-o usando o caminho ./data/nomedoarquivo.

* *URL:* Se o conjunto de dados estiver disponível em uma URL, inclua a URL no seu código e carregue-o diretamente.

**2. Separação dos dados por cliente:**

* Para simular 4 clientes, divida os dados em 4 partes.

**3. Crie uma instância para IRISDataset, IRISModel, IRISExperiment.**

**4. Execute as funções:**
* ***generate_server_fn:*** Cria a estratégia para o aprendizado federado
* ***generate_client_fn:*** Gera o modelo e o dataset de cada cliente.
* ***evaluate_fn:*** Avalia o modelo global usando o dataset de um dos clientes.
* ***run_federated_2:*** Executa o experimento federado.

In [ ]:
from flautim.pytorch.common import run_federated, weighted_average
from flautim.pytorch import Model, Dataset
from flautim.pytorch.federated import Experiment
import IRISDataset, IRISModel, IRISExperiment
import flautim as fl
import flwr
from flwr.common import Context, ndarrays_to_parameters
from flwr.server import ServerConfig, ServerAppComponents
import pandas as pd
import numpy as np


def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
    }
    return config



def generate_server_fn(context, eval_fn, **kwargs):

    def create_server_fn(context_flwr:  Context):

        net = IRISModel.IRISModel(context, suffix = 0)
        params = ndarrays_to_parameters(net.get_parameters())

        strategy = flwr.server.strategy.FedAvg(
                          initial_parameters=params,
                          evaluate_metrics_aggregation_fn=weighted_average,
                          fraction_fit=0.2,  # 10% clients sampled each round to do fit()
                          fraction_evaluate=0.5,  # 50% clients sample each round to do evaluate()
                          evaluate_fn=eval_fn,
                          on_fit_config_fn = fit_config,
                          on_evaluate_config_fn = fit_config
                          )
        num_rounds = 20
        config = ServerConfig(num_rounds=num_rounds)

        return ServerAppComponents(config=config, strategy=strategy)
    return create_server_fn

def generate_client_fn(context, files):

    def create_client_fn(context_flwr:  Context):

        cid = int(context_flwr.node_config["partition-id"])
        file = int(cid)
        model = IRISModel.IRISModel(context, suffix = cid)
        dataset = IRISDataset.IRISDataset(files[file], batch_size = 10, shuffle = False, num_workers = 0)

        return IRISExperiment.IRISExperiment(model, dataset, context).to_client()

    return create_client_fn


def evaluate_fn(context, files):
    def fn(server_round, parameters, config):
        """This function is executed by the strategy it will instantiate
        a model and replace its parameters with those from the global model.
        The, the model will be evaluate on the test set (recall this is the
        whole MNIST test set)."""

        model = IRISModel.IRISModel(context, suffix = "FL-Global")
        model.set_parameters(parameters)

        dataset = IRISDataset.IRISDataset(files[0], batch_size = 10, shuffle = False, num_workers = 0)

        experiment = IRISExperiment.IRISExperiment(model, dataset, context)

        config["server_round"] = server_round

        loss, _, return_dic = experiment.evaluate(parameters, config)

        return loss, return_dic

    return fn

if __name__ == '__main__':

    context = fl.init()
    fl.log(f"Flautim2 inicializado!!!")

    num_clientes = 2

    iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
    iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
    iris['class'] = pd.factorize(iris['class'])[0]


    iris = iris.sample(frac=1, random_state=42).reset_index(drop=True)
    files = np.array_split(iris, num_clientes)

    client_fn_callback = generate_client_fn(context, files)
    evaluate_fn_callback = evaluate_fn(context, files)
    server_fn_callback = generate_server_fn(context, eval_fn = evaluate_fn_callback)

    run_federated(client_fn_callback, server_fn_callback)